# Lab 3 - Time-dependence of RC, RL, and RLC circuits
### PHYS 222, Spring 2024, Dr. Ben Zwickl, Dr. Ke Xu, Dr. Michael Pierce

### General rules
* Any code, calculations, or figures you create in Python should have screenshots in the OneNote notebook. 
* If you need to do handwritten calculations, include a photo of those in OneNote. 
* You may use Colab or a local Jupyter installation. Jupyter is strongly encouraged for the way it lets you split a calculation into several cells. You should add new cells after each Task prompt. 

### Goals
* **Measure the response of RC, RL and RLC**
    * Choosing R values and time scales
    * Observing the transition from underdamped to overdamped response
* **Fit data to a theoretical model** 
    * Defining model functions
    * Subsetting data for fitting
    * Using `scipy.optimize.curve_fit` 
    * Measuring capacitance and inductance using a fit
* **Improve your `load_waveform` function that automates the data import and plotting process**
    * Option to produce new plot or use existing plot

In [ ]:
# load packages at the top of your notebook. 
# We will always use numpy and matplotlib.pyplot.
# In future weeks, you should add these lines yourself

import numpy as np
import matplotlib.pyplot as plt

# Task 1: Add a new feature to the `load_waveform` function (1 pt)


Starting with copying and pasting your code from the previous lab, update the `load_waveform` function to include a new input.  You can do this with modifying one line of code (the `def` line) and adding two lines of code.  

**Function name:** `load_waveform`

**Input parameters:** 
* `filename` - filename for a CSV file saved from the M2K with 2 channels of data
* `newfig` - a boolean (True/False) of whether or not a new figure should be produced each time the funtion is called. Default is `True`

**Output:** 
* Display a plot of the loaded data, including axes labels and legend
* Returns `[t,ch1,ch2]` - a list of 3 arrays representing time, CH1 volts, and CH2 volts

**Need help** on making a new figure in matplotlib? Use `plt.figure()` to [create a new figure](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.figure.html). 

**Need help** on making default values for parameters in a function definition? See this [short example](https://www.w3schools.com/python/gloss_python_function_default_parameter.asp).

**Insert cell or cells below to record your answers**

In [ ]:
# RUN THESE TEST CASES BUT DON'T EDIT THEM

### Test cases for load_waveform
path_testdata = "https://github.com/bzwickl/PHYS222_SP24_Student/raw/main/Lab3/Sample_Scopy_Data.csv"
data = load_waveform(path_testdata)
assert len(data) == 3        # make sure the function returns a list of 3 things
t, ch1, ch2 = data            # unpack for analysis
assert len(t) == len(ch1) == len(ch2) == 3200  # check length of imported data set
assert abs(ch1.mean() - -0.0033719303437499894)/-0.0033719303437499894 < 1e-4  # check the values on ch1
assert (np.mean( np.abs(ch2) ) - 0.5011812406249)/ 0.5011812406249 < 1e-4  # check the values on ch2
assert abs( (t.min() - (-0.0016)) ) /abs(-0.0016) < 1e-4   # check the min time

# Task 2: Build an RC circuit, collect data, and plot (1 pt)
Task 2 asks you to build a circuit and take data in way very similar to Lab 2. 
<img src="RC_circuit.PNG" alt="Diagram for RC circuit"> 
You should produce a waveform similar to this, where Blue (Ch2. 2+) is measuring the voltage of the output of the signal generator (W1, Yellow) and Orange (Ch1, 1+) is measuring the voltage across the capacitor. 

1. Build the RC circuit on your breadboard.
1. Use the Waveform (Signal) Generator (W1) as the voltage source
    * Set to a **square wave**. 
    * An amplitude of around 1 V is fine
1. Use both channels of the oscilloscope to simultaneously measure voltages at nodes $A$ and $B$.
1. **Set up the oscilloscope trigger so that $t = 0$ occurs when the square wave switches from high to low.** This will make your subsequent analysis MUCH easier because $t = 0$ will be the start of the discharge. 
1. Save your data. You may analyze on Google Drive and Colab or on a local Python installation, whichever you prefer. 
1. Plot the data using your `load_waveform` function. Make sure you have a clear view of at least 1 of the expoential decays.

Include photos or screenshots in your OneNote notebook showing: 
* The circuit diagram
* A photo of the built circuit
* Your plot of data from Python

# Task 3: Define a fit function (2 pts)
We need to define a Python function that represents the fit. 

The transient response to a step function is given by an exponential decay.

$$V(t) = V_0 e^{-t/\tau} + V_\textrm{offset}$$.

Define the following function:

**Function name:** `exp_decay`

**Inputs** 
* `t` - time (s)
* `V0` - initial voltage at time = 0
* `tau` - decay time
* `Voffset` - offset voltage that remains after the exponential decay

**Outputs**
* `V`  - current position at time `t` 

In [ ]:
### TEST CASES for exp_decay function
# If the cell runs without any error messages, you passed the test
assert ( exp_decay(0,5,2,3.3) - 8.3 )/8.3 < 1e-4   
assert ( exp_decay(4,5,2, 3.3)  - 3.97667 )/3.97667 < 1e-4   

# Task 4: Fit the data for the decay time and estimate the capacitance (2 pts)

You will use `scipy.optimize.curve_fit` to find the best fit parameters that provide the closest match between the model and your data. Review the [official documentation for curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) for how to use `curve_fit` or see the video linked below.

1. Use the line `from scipy.optimize import curve_fit` to import the function.
1. Subset the data to include only a single exponential decay (see tutorial link below).
1. Use `curve_fit` and your `exp_decay` function to find the optimal parameters. (see tutorial link below)
1. Make a plot overlaying the best fit and the data. You should see a fantastically good fit.
1. Estimate the capacitance (nominal 10 nF) based on the relationship between $R$, $C$, and the decay time $\tau$.

**Need help?** 
* [Video tutorial on taking a subset of data](https://youtu.be/FvC_jfoR3DI) using indices or (even better) a Boolean condition array
* [Video tutorial on use Scipy's `curve_fit` function](https://youtu.be/5dT6Oauy5-Q).


## Comment on the M2K Oscilloscope
The following specifications are from the [M2K's official documentation](https://wiki.analog.com/university/tools/m2k).

Here is a brief meaning of the specifications.
* **12-bit** refers to the number of voltage steps between the min value and max value (usually as seen on screen). For example, if the scope is on the 1 V/div scale, then the full range (ten dividions) is 10 V from min to max.  This 10 V full range is divided into $2^{12}$ steps. Each step is $10\ \textrm{V}/\ 4096 \approx 2.44 mV$.  However, changing to a 100 mV/div scale will make the  full range span only 1 V, so the step size decreases by 10x, and would be 0.244 mV. 
* **100 MS/s sample rate** means the scope takes at most 100 million voltage samples per second. So a 10 MHz signal would get sampled 10 times per oscillation, which does not produce a very accurate waveform. Such a coarsely sampled waveform can still be sometimes useful, but most of the time we will measure signals under 10 MHz. 
* **10 MHz bandwidth (maybe higher)** means the circuits in the M2K don't respond well above 10 MHz. It usually isn't a sharp cutoff, but a gradual decline in response. So a 2 V signal at 10 MHz may not appear to be 2 MHz, but slightly smaller.
* **1 MOhm input impedance** is a spec we encountered in lab 1. When connecting the the 1+/1- inputs from the M2K, it acts like a 1 MOhm resistor is connected in parallel with whatever circuit is being measured.

# Task 5: Build the RL circuit, plot, fit to find $L$ (2 pts)

![LR Circuit Diagram](LR_circuit.PNG)

The nominal inductance is $L = 10$ mH. We can use measurements of the transient response of the RL circuit to estimate the inductance, $L$. Recall if a step function or square wave is applied to the RL circuit, the resopnse is an exponential  response with a decay time given by 

$$\tau_\textrm{RL} = \frac{L}{R}$$

1. Measure the the resistance of the inductor, $R_L$. It is not zero and can't be ignored. Use the DMM for the resistance measurement.
1. Pick a value for $R$ that should produce a measurable ringdown time. Because of the oscilloscope's bandwidth of 10 MHz, you definitely want to make sure $\tau_\textrm{RL} >> 1/(10 \textrm{ MHz}) = 100 \textrm{ ns}$.  By "much greater than" it means several orders of magnitude higher.  
1. Similar to task 3, use both oscilloscope channels to record the square wave and the voltage across the inductor.
1. **Set up the oscilloscope trigger so that $t = 0$ occurs when the square wave switches between low and high.** This will make your subsequent analysis MUCH easier because $t = 0$ will be the start of the exponential decay. 
1. Save a data set and plot in your Jupyter notebook.
1. Fit the exponential decay data, add your fit to the graph with the data.  Don't forget to subset the data before fitting.
1. What value of $L$ do you infer from your value of $\tau$?
1. Why doesn't the voltage across the inductor decay all the way to zero?

# Task 6 RLC transients and critical damping (2 pts)
This task is an experimental exploration of RLC circuit response to a step function change in the voltage. You will be able to observe the transition from underdamped oscillations to an overdamped decay. 
![Circuit diagram for LRC circuit](LRC_circuit.PNG)
* $L_1$ - 10 mH inductor
* $C_1$ - 10 nF capacitor
* $R_1$ - vary the value to observe transition from underdamping to critical damping to overdamping. 

Condition for critical damping: 
$$ 4LC -R^2 C^2 = 0$$.

Equation for oscillation frequency during underdamping 
$$f=\frac{1}{2\pi}\frac{\sqrt{4LC - R^2 C^2}}{2LC}$$

### Example plot of what you might see going from underdamping to overdamping
![underdamping to overdamping transition](underdamping_to_overdamping.PNG)

1. What resistance value $R$ should create critical damping? (show calculations in your Jupyter notebook in a code cell)
1. What frequency would the circuit oscillate at if there no damping ($R = 0$)? 
1. Use at least 5 different resistance values to explore the transition from underdamping to overdamping. 
1. Do the theoretical equations for the critical damping condition and the oscillation frequency match your data? (They should)
1. If possible, use subplots to arrange the plots in a compact arrangement. 